
# Trackintel case study - Tracking dataset comparison
This notebook presents a case-study to jointly analyse four different tracking datasets using [trackintel](https://github.com/mie-lab/trackintel). 
Three of the datasets that are used in this case study can not be published to protect the privacy of the participants. You can find an executable example notebook for trackintel here: [binder](https://mybinder.org/v2/gh/mie-lab/trackintel/HEAD?urlpath=https%3A%2F%2Fgithub.com%2Fmie-lab%2Ftrackintel%2Fblob%2Fmaster%2Fexamples%2Ftrackintel_basic_tutorial.ipynb) [code](https://github.com/mie-lab/trackintel/blob/master/examples/trackintel_basic_tutorial.ipynb)

## Imports

In [2]:
import os
from collections import defaultdict
import pickle

import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

import psycopg2
import json
import seaborn as sns
from shapely.geometry import Point

import trackintel as ti
from trackintel.analysis.tracking_quality import temporal_tracking_quality
from trackintel.analysis.modal_split import calculate_modal_split
from trackintel.visualization.modal_split import plot_modal_split

In [4]:
out_path = "./temp"
os.makedirs(out_path, exist_ok=True)

path_to_geolife = r"E:\Geolife Trajectories 1.3\Data_10"
DBLOGIN_FILE = os.path.join("../mobility-graph-representation/dblogin.json")
DBLOGIN_FILE = r"C:\Users\henry\OneDrive\Programming\21_mobility-graph-clustering\dblogin.json"
DBLOGIN_FILE_GC = r"C:\Users\henry\OneDrive\Programming\21_mobility-graph-clustering\dblogin_source.json"
DBLOGIN_FILE_yumuv = r"C:\Users\henry\OneDrive\Programming\21_mobility-graph-clustering\dblogin_mielab.json"

with open(DBLOGIN_FILE) as json_file:
    LOGIN_DATA = json.load(json_file)

with open(DBLOGIN_FILE_GC) as json_file:
    LOGIN_DATA_GC = json.load(json_file)

with open(DBLOGIN_FILE_yumuv) as json_file:
    LOGIN_DATA_YUMUV = json.load(json_file)
    
con = psycopg2.connect(
        dbname=LOGIN_DATA["database"],
        user=LOGIN_DATA["user"],
        password=LOGIN_DATA["password"],
        host=LOGIN_DATA["host"],
        port=LOGIN_DATA["port"],
)

con_gc = psycopg2.connect( # requires ssh tunnel
        dbname=LOGIN_DATA_GC["database"],
        user=LOGIN_DATA_GC["user"],
        password=LOGIN_DATA_GC["password"],
        host=LOGIN_DATA_GC["host"],
        port=LOGIN_DATA_GC["port"],
)

con_yumuv = psycopg2.connect( # requires ssh tunnel
        dbname=LOGIN_DATA_YUMUV["database"],
        user=LOGIN_DATA_YUMUV["user"],
        password=LOGIN_DATA_YUMUV["password"],
        host=LOGIN_DATA_YUMUV["host"],
        port=LOGIN_DATA_YUMUV["port"],
)

CRS_WGS84 = "epsg:4326"

In [3]:
studies = ["gc1", "gc2", "yumuv_graph_rep", "geolife"]

# for plotting later:
study_mapping = {"gc1":"Green Class 1", "gc2": "Green Class 2", "yumuv_graph_rep": "Yumuv", "geolife":"Geolife"}

## Datasets
We include the data from four tracking studies with two different tracking data types.

### Green Class 1 & 2 
The Green Class 1 & 2 studies were conducted in collaboration with the Swiss Federal Railway Systems (SBB) under the project name [SBB Green Class](https://www.researchgate.net/publication/335858551_Analysis_framework_and_results_of_the_SBB_Green_Class_pilot_studies)
In both studies, participants were given full access to all public transport in Switzerland. In addition, the participants from the first Green Class study (Green Class 1) received an electric vehicle and the ones from the second study (Green Class 2) an e-bike. Study participants were tracked with a GNSS-based application (app) that provides partially preprocessed data as staypoints and triplegs.
### Geolife
The first study is the open-source [Geolife dataset](https://www.microsoft.com/en-us/download/details.aspx?id=52367&from=https%3A%2F%2Fresearch.microsoft.com%2Fen-us%2Fdownloads%2Fb16d359d-d164-469e-9fd4-daa38f2b2e13%2F) that covers the movement of employees of Microsoft Research Asia. The dataset is from about 2012 and was still recorded with dedicated GPS-only trackers. As the study took place in an urban area the GPS was unreliable and there are many gaps in the dataset. However, Geolife is still one of the few publicly available tracking datasets.

### Yumuv
The [yumuv](https://www.research-collection.ethz.ch/handle/20.500.11850/521380) study investigated the impact of a [Mobility-as-a-Service app](https://yumuv.ch/en) that integrates shared e-scooters, e-bikes and public transport. In the yumuv study, participants were divided into control and treatment group and were tracked for three months using an app that already provides staypoints and triplegs.


# Import data
One main advantage of trackintel is its ability to standardize the preprocessing of tracking data. For this, the different datasets have to be imported to geopandas dataframes that fulfill the requirements described [here](https://trackintel.readthedocs.io/en/latest/modules/model.html)

The trackintel data model consists of these different classes:

- positionfixes [pfs]: Raw GPS data.
- staypoints [sp]: Locations where a user spent a minimal time.
- triplegs [tpls]: Segments covered with one mode of transport.
- locations [loc]: Clustered staypoints.
- trips: Segments between consecutive activity staypoints (special staypoints that are not just waiting points).
- tours: Sequences of trips which start and end at the same location (if the column ‘journey’ is True, this location is home).

The Geolife dataset consists of only positionfixes while the other datasets are already processed to staypoints and triplegs by the tracking app. We will now import all datasets and combine them on the staypoints / triplegs level


## Import Geolife
Geolife is an important benchmark dataset. Trackintel therefore offers a dedicated function that reads in the Geolife dataset. The import function takes care of the required column format and names.


In [4]:
from trackintel.io.dataset_reader import read_geolife
pfs_geolife, _ = read_geolife(path_to_geolife, print_progress=True)
pfs_geolife.head(3)

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:13<00:00,  1.38s/it]


,elevation,tracked_at,geom,user_id,accuracy
id,,,,,
0,149.9616,2008-10-23 02:53:04+00:00,POINT (116.31842 39.98470),0,NaN
1,149.9616,2008-10-23 02:53:10+00:00,POINT (116.31845 39.98468),0,NaN
2,149.9616,2008-10-23 02:53:15+00:00,POINT (116.31842 39.98469),0,NaN


### Generate staypoints
We generate staypoints from the positionfixes and define all staypoints as relevatn activities if they are longer than 25 minutes.

In [5]:
# extract staypoints
pfs_geolife, sp_geolife = pfs_geolife.as_positionfixes.generate_staypoints(
    gap_threshold=24 * 60, include_last=True, print_progress=True, dist_threshold=200, time_threshold=30, n_jobs=4
)

# add activity flag to staypoints
sp_geolife = sp_geolife.as_staypoints.create_activity_flag(method="time_threshold", time_threshold=25)
sp_geolife.head(3)

C:\Users\henry\.conda\envs\geodev\lib\site-packages\trackintel\preprocessing\positionfixes.py:111: UserWarning: 1686 duplicates were dropped from your positionfixes. Dropping duplicates is recommended but can be prevented using the 'exclude_duplicate_pfs' flag.
  warnings.warn(warn_str)
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:05<00:00,  1.90it/s]


,user_id,started_at,finished_at,elevation,geom,is_activity
id,,,,,,
0,0,2008-10-23 03:03:45+00:00,2008-10-23 04:08:07+00:00,61.7220,POINT (116.29917 39.98341),True
1,0,2008-10-23 04:32:52+00:00,2008-10-23 09:42:25+00:00,53.9496,POINT (116.32451 39.99967),True
2,0,2008-10-23 11:10:42+00:00,2008-10-24 02:10:09+00:00,26.8224,POINT (116.32105 40.00917),True


### Generate triplegs

In [6]:
%%time
pfs_geolife, tpls_geolife = pfs_geolife.as_positionfixes.generate_triplegs(sp_geolife, method="between_staypoints", gap_threshold=25)
tpls_geolife.head(3)

Wall time: 25 s


,user_id,started_at,finished_at,geom
id,,,,
0,0,2008-10-23 02:53:04+00:00,2008-10-23 03:03:40+00:00,"LINESTRING (116.31842 39.98470, 116.31845 39.9..."
1,0,2008-10-23 04:08:07+00:00,2008-10-23 04:32:47+00:00,"LINESTRING (116.28680 39.99578, 116.28545 39.9..."
2,0,2008-10-23 09:42:25+00:00,2008-10-23 11:10:37+00:00,"LINESTRING (116.32016 40.00478, 116.32039 40.0..."


#### Included tracking studies

## Import Green Class 1 & 2
The other datasets are stored in a postgis database and were already preprocessed to staypoints (static behavior) and triplegs (movement). To use them with trackintel, the data has to be adjusted to the data model. This means to create a geodataframe with the correct column names and timezone aware timestamps.


### Staypoints

In [7]:
sp_list = []
for study in ['gc1', 'gc2']:
    # download data
    sp_temp = gpd.GeoDataFrame.from_postgis(sql=f"SELECT * FROM {study}.staypoints",
        con=con_gc, geom_col="geometry_raw", index_col="id")
    
    # transform to trackintel dataframe
    sp_temp = ti.io.read_staypoints_gpd(sp_temp, geom_col='geom', tz='UTC', crs=CRS_WGS84,
                                                 mapper={"geometry_raw": "geom", "purpose_validated": "activity_label"})

    # green class specific definition of activities
    sp_temp = sp_temp.as_staypoints.create_activity_flag(method="time_threshold", time_threshold=25)
    meaningful_purpose = ~sp_temp["activity_label"].isin(["wait", "unknown"])
    sp_temp.loc[meaningful_purpose, 'is_activity'] = True
    
    # keep study as attribute
    sp_temp['study'] = study_mapping[study]
    
    sp_list.append(sp_temp)


### triplegs

In [8]:
tpls_list = []
for study in ['gc1', 'gc2']:
    # downlaod data
    tpls_temp = gpd.GeoDataFrame.from_postgis(
        sql=f"SELECT * FROM {study}.triplegs where ST_isValid(geometry) limit 1000",con=con_gc,
        crs=CRS_WGS84, geom_col="geometry", index_col="id")
    # transform to trackintel dataframe
    tpls_temp = ti.io.read_triplegs_gpd(tpls_temp, geom_col='geom',
        crs=CRS_WGS84,
        tz='UTC',
        mapper={"geometry": "geom"})

    tpls_list.append(tpls_temp)

## Import yumuv

### staypoints

In [12]:
sp_yumuv = ti.io.read_staypoints_postgis("select * from yumuv.staypoint limit 100", con=con_yumuv, geom_col='geometry', 
                                         crs=CRS_WGS84, index_col="id", tz="UTC",
                                        mapper={'user_fk': 'user_id'})

sp_yumuv = sp_yumuv.rename({'geometry': 'geom'}, axis=1).set_geometry('geom')
sp_yumuv['study'] = 'yumuv'
sp_yumuv = sp_yumuv.as_staypoints.create_activity_flag(time_threshold=25)

### triplegs

In [13]:
tpls_yumuv = ti.io.postgis.read_triplegs_postgis("select * FROM yumuv_graph_rep.triplegs", con=con_yumuv, geom_col='geometry', 
                                                 crs=CRS_WGS84, index_col="id", tz="UTC", 
                                                )
tpls_yumuv = tpls_yumuv.rename({'geometry': 'geom'}, axis=1).set_geometry('geom')


ValueError: Query missing geometry column 'geometry'

In [ ]:
geom_not_valid = ~tpls.geometry.is_valid
print("invalid triplegs", sum(geom_not_valid))

In [10]:
valid_tstamp_flag_sp = sp_gc1.started_at <= sp_gc1.finished_at
valid_tstamp_flag_tpls = sp_gc2.started_at <= sp_gc2.finished_at
print((~ valid_tstamp_flag_sp).sum(), (~valid_tstamp_flag_tpls).sum())

NameError: name 'sp_gc1' is not defined

In [ ]:
df_temp_list = []
for ix, study in enumerate(studies):
    sql = f"SELECT * FROM {study}.staypoints order by started_at"
    df_temp = ti.io.read_staypoints_postgis(sql, con, index_col="id", geom_col="geom")
    
    df_temp['study'] = study_mapping[study]
    df_temp.index = df_temp.index + ix * 10e7 # ensure unique index over studies
    
    df_temp.rename({'purpose': 'activity_label', 'activity': 'is_activity'}, axis=1, inplace=True)
    if study in ['gc1', 'gc2']:
        df_temp['activity_label'] = df_temp['activity_label'].apply(eval)
        #print(study, df_temp.activity_label.unique())
        
    df_temp_list.append(df_temp)

staypoints = pd.concat(df_temp_list)

In [ ]:
# flatten activity label (gc1 preprocessing)
def flatten_activity(activity_label):
    if isinstance(activity_label, list):
        if 'home' in activity_label:
            return 'home'
        elif 'work' in activity_label:
            return 'work'
        else:
            return activity_label[0]
    else:
        return activity_label
    
staypoints['activity_label'] = staypoints['activity_label'].apply(flatten_activity)


In [ ]:
staypoints

In [ ]:
user_study_matching = staypoints[['user_id', 'study']].drop_duplicates()
user_study_matching = user_study_matching.set_index('user_id')

### Merge triplegs

#### Todo: 
- drop additional columns
- name mode column `mode` for all datasets

In [ ]:
# get triplegs
df_temp_list = []
for ix, study in enumerate(studies):
    print(study)
    sql = f"SELECT * FROM {study}.triplegs order by started_at"
    df_temp = ti.io.read_triplegs_postgis(sql, con, index_col="id", geom_col="geom")
    df_temp['study'] = study_mapping[study]
    df_temp.index = df_temp.index + ix * 10e7 # ensure unique index over studies
    df_temp.rename({'mode_validated': 'mode', 'track_mode_corrected': 'mode'}, axis=1, inplace=True)
    df_temp_list.append(df_temp)

triplegs = pd.concat(df_temp_list)


### create trips

In [ ]:
staypoints, triplegs, trips = ti.preprocessing.triplegs.generate_trips(staypoints=staypoints, triplegs=triplegs,
                                                                       gap_threshold=15, add_geometry=True)

### create tours

In [ ]:
trips, tours = ti.preprocessing.generate_tours(trips, staypoints=staypoints)

In [ ]:
trips = trips.join(user_study_matching, on='user_id')
tours = tours.join(user_study_matching, on='user_id')

### create locations

In [ ]:
staypoints, locations = ti.preprocessing.generate_locations(staypoints, epsilon=30, num_samples=1, 
                                                distance_metric='haversine', print_progress=True,
                                                n_jobs=4)

In [ ]:
locations = locations.join(user_study_matching, on='user_id')

### statistics

In [ ]:
overview_df = pd.DataFrame(index=study_mapping.values())

nr_user = trips['user_id'].nunique()
overview_df['nr_user'] = nr_user

# nb staypooints
overview_df['nr_sp'] = staypoints.groupby(by=['study']).size()
overview_df['nr_staypoints_per_user'] = overview_df['nr_sp']/nr_user

# nb triplegs
overview_df['nr_tpls'] = triplegs.groupby(by=['study']).size()
overview_df['nr_triplegs_per_user'] = overview_df['nr_tpls']/nr_user

# nb trips
overview_df['nr_trips'] = trips.groupby(by=['study']).size()
overview_df['nr_trips_per_user'] = overview_df['nr_trips']/nr_user
# nb locations

overview_df['nr_loc'] = locations.groupby(by=['study']).size()
overview_df['nr_locations_per_user'] = overview_df['nr_loc']/nr_user

# nb tours
overview_df['nr_tours'] = tours.groupby(by=['study']).size()
overview_df['nr_tours_per_user'] = overview_df['nr_tours']/nr_user


In [ ]:
overview_df

## 3) Add further details

* How many trips comprise one tour on average? How many triplegs comprise a trip?
* What is the average trip duration / distance?

#### Number of hops in tours

In [ ]:
tours['nb_hops'] = tours["trips"].apply(lambda x: len(x))
overview_df['avg_hops_tours'] =  tours.groupby('study')['nb_hops'].mean() 

#### Trip duration

In [ ]:
 (trips['finished_at'] - trips['started_at']).dt.total_seconds() / 3600

In [ ]:
trips['duration_s'] = (trips['finished_at'] - trips['started_at']).dt.total_seconds() / 3600
overview_df[["mean_trip_dur", "std_trip_dur"]] = trips.groupby('study')['duration_s'].aggregate(['mean', 'std'])
overview_df = overview_df.round({'mean_trip_dur': 2, 'std_trip_dur': 2})
overview_df

#### Overall tracking period and number of trips per day per user

In [ ]:
min_max_times_by_user = trips.groupby(['study', 'user_id']).aggregate({'started_at': 'min', 'finished_at': 'max'})
duration_by_user = (min_max_times_by_user['finished_at'] - min_max_times_by_user['started_at']).dt.total_seconds() / (3600 * 24)
overview_df[["mean_tracking_period", "std_tracking_period"]] = duration_by_user.groupby('study').aggregate(['mean', 'std'])
overview_df

In [ ]:
# average trips per day on days with data
trips_temp = trips.set_index('started_at')
nr_trips_per_user_per_day = trips_temp.groupby(['study', 'user_id', pd.Grouper(freq='D')]).size().groupby('study').mean() 
overview_df['nr_trips_per_user_per_day'] = nr_trips_per_user_per_day

In [ ]:
# average trips per day over full tracking period
nr_trips_per_user_per_day = (trips.groupby(['study', 'user_id']).size() / duration_by_user).groupby('study').mean()

# there are some outliers that have tracked <<< 1 day (due to the limit set by me)

#### Trip distance

In [ ]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', None)
#triplegs.geometry

In [ ]:
#ti.preprocessing.smoothen_triplegs(triplegs, tolerance=0.1, preserve_topology=False)



In [ ]:
triplegs['length_in_m'] = ti.geogr.distances.calculate_haversine_length(triplegs)


In [ ]:
metrics_per_trip = triplegs.groupby(['study', 'trip_id']).agg({"trip_id": "count", "length_in_m":"sum"})
overview_df[["nr_legs_per_trip", "avg_trip_length", "std_trip_length"
            ]] = metrics_per_trip.groupby('study').agg({'trip_id': 'mean', 'length_in_m': ["mean", "std"]})

#### Tracking quality

In [ ]:
triplegs_staypoints = pd.concat([triplegs, staypoints])
tracking_quality_by_user_study = triplegs_staypoints.groupby('study').apply(temporal_tracking_quality)
overview_df[["mean_tracking_quality", "std_tracking_quality"]] = tracking_quality_by_user_study.groupby('study').agg(['mean', 'std'])


## 4) Finalize results in two tables

In [ ]:
# Make DataFrame from out_dict and save to csv
#df = pd.DataFrame(out_dict).swapaxes(1,0)
#df.to_csv(os.path.join(out_path, "trackintel_casestudy_df.csv"))

In [ ]:
# load df
#df = pd.read_csv(os.path.join(out_path, "trackintel_casestudy_df.csv"), index_col="Unnamed: 0")

#### 4.1) Table with simple dataset stats

In [ ]:
overview_df[["nr_user","mean_tracking_period", "std_tracking_period", "nr_loc", "nr_sp", "nr_tpls", "nr_trips", "nr_tours"]]
overview_df.fillna(0, inplace=True)
overview_df

In [ ]:
# basic info
df_basic = overview_df[["nr_user","mean_tracking_period", "std_tracking_period", "nr_loc", "nr_sp", "nr_tpls", "nr_trips", "nr_tours"]].astype(int)

In [ ]:
df_basic["Tracking period (std)"] = df_basic.apply(lambda x: f"{x['mean_tracking_period']} ({x['std_tracking_period']})", axis=1)
df_basic["Available"] = "Staypoints, Triplegs"
df_basic["Study type"] = "GNSS (app)"



# rename columns and index
df_basic = df_basic.rename(columns={"Available": "Input", "nr_user": "Users", "nr_loc": "Locations", "nr_sp": "Staypoints", "nr_tpls": "Triplegs", 
                        "nr_trips": "Trips", "nr_tours": "Tours"}, 
                        index=study_mapping).drop(columns=["mean_tracking_period", "std_tracking_period"])
# select the columns in the right order
df_basic = df_basic[["Users", 'Tracking period (std)' , "Input", "Study type", "Locations", "Staypoints", "Triplegs", "Trips", "Tours"]]

df_basic
# print(df_basic.to_latex())

#### 4.2) Table with analysis

In [ ]:
# Analysis table
detailed = overview_df[["nr_trips_per_user_per_day", "mean_trip_dur", "std_trip_dur", "avg_hops_tours", "nr_legs_per_trip", "avg_trip_length", "std_trip_length", "mean_tracking_quality", "std_tracking_quality"]]
detailed = detailed.round(2)
# Incorporate mean and std in one cell
detailed["Trip distance (std)"] = detailed.apply(lambda x: f"{int(x['avg_trip_length'])} ({int(x['std_trip_length'])})", axis=1)
detailed["Trip duration (std)"] = detailed.apply(lambda x: f"{x['mean_trip_dur']} ({x['std_trip_dur']})", axis=1)
detailed["Tracking quality (std)"] = detailed.apply(lambda x: f"{x['mean_tracking_quality']} ({x['std_tracking_quality']})", axis=1)

detailed = detailed.rename(columns={"nr_trips_per_user_per_day": "Trips per day", 
                                   "avg_hops_tours": "Trips per tour", 
                                   "nr_legs_per_trip": "Legs per trip"}, 
                          index={"gc1":"Green Class 1", "gc2": "Green Class 2", "yumuv_graph_rep": "Yumuv", "geolife":"Geolife"}
                          ).drop(["avg_trip_length", "std_trip_length", "mean_tracking_quality", "std_tracking_quality",
                                  "mean_trip_dur", "std_trip_dur"
                                 ], axis=1)

detailed
# print(detailed.to_latex())

## 5) KDE plot for user tracking quality distribution

In [ ]:
new_df = tracking_quality_by_user_study.reset_index(drop=False, level=0).drop('user_id', axis=1)
new_df = new_df.pivot(columns='study', values='quality')
new_df

In [ ]:
#for key in ["gc1","gc2",  "geolife", "yumuv_graph_rep"]:
 #   study = study_mapping[key]
  #  new_df[study] = tracking_quality_dfs[key]["quality"]

plt.figure(figsize=(5,3))
plt.rcParams.update({"font.size": 15})
g = sns.kdeplot(data=new_df, fill=True, alpha=.3, legend=True, common_grid=True)
# sns.histplot(data=new_df, bins=100,common_norm=True, stat="density", kde=False)
plt.ylabel("Density (users)")
plt.xlabel("Tracking quality")
sns.move_legend(g, "upper left")
plt.xlim(0, 1)
#plt.savefig(os.path.join(out_path, "tracking_quality_kde.png"))
plt.show()

## 6) Modal split dataset comparison

#### Main plotting function to compare the modal split of several studies

#### Apply Trackintel function to get the modes for Geolife

In [ ]:
is_geolife = triplegs['study'] == 'Geolife'
tpls_geolife = triplegs[is_geolife]
tpls_geolife

In [ ]:

is_geolife = triplegs['study'] == 'Geolife'


triplegs[is_geolife] = ti.analysis.labelling.predict_transport_mode(triplegs[is_geolife])


In [ ]:
# we need to map the mode labels from GC1 / GC2 / Yumuv to the ones from geolife
mode_to_category = {
    'Mode::Bicycle': 'slow',
     'Mode::Car': 'motorized',
     'Mode::Train': 'fast',
    'Mode::Walk': 'slow',
    'Mode::Tram': 'motorized',
    'Mode::Bus': 'motorized',
    'Mode::Airplane': 'fast',
    'Mode::Coach': 'motorized',
    'Mode::Ebicycle': 'slow',
    'Mode::Boat': 'slow',
    'Mode::Ecar': 'motorized',
    "fast_mobility": "fast",
    "motorized_mobility": "motorized",
    "slow_mobility": "slow",
    "airplane": "fast", 
    "bicycle": "slow",
    "boat": "slow",
    "bus": "motorized",
    "car": "motorized",
    "coach": "motorized",
    "ebicycle": "slow",
    "kick_scooter": "slow",
    "ecar": "motorized",
    "motorbike": "motorized",
    "ski": "slow",
    "train": "fast",
    "tram": "motorized",
    "walk": "slow",
    "fast": "fast",
    "motorized": "motorized",
    "slow": "slow"
}

triplegs['mode'] = triplegs['mode'].map(mode_to_category)
triplegs

In [ ]:
triplegs.dropna(subset=['mode'], inplace=True)
triplegs

In [ ]:
# count
modal_split_count = triplegs.groupby('study').apply(calculate_modal_split, metric='count', norm=True)
modal_split_count = modal_split_count.droplevel(1)
modal_split_count

In [ ]:
# duration
modal_split_duration = triplegs.groupby('study').apply(calculate_modal_split, metric='duration', norm=True)
modal_split_duration = modal_split_duration.droplevel(1)
modal_split_duration

In [ ]:
# modal_split = triplegs.groupby('study').apply(calculate_modal_split, metric='distance', norm=True) https://github.com/geopandas/geopandas/issues/1777
modal_split_list = []
for study, tpls in triplegs.groupby('study'):
    modal_split_distance = calculate_modal_split(tpls, metric='distance', norm=True)
    modal_split_distance['study'] = study
    modal_split_list.append(modal_split_distance)
    
modal_split_distance = pd.concat(modal_split_list).set_index('study')
modal_split_distance
    

### Get modal split for each study

### Plotting

In [ ]:
triplegs['mode'].unique()

In [ ]:

triplegs

# extent mode_to_category mapping with the upper case / colon versions
#further_cols = list(modal_split_count["gc1"].columns)
#for col in further_cols:
#    mode_to_category[col] = mode_to_category[col.split("::")[1].lower()] 

In [ ]:
colors = ['#1D2F6F', '#8390FA', '#6EAF46', '#FAC748']
color = dict(zip(['fast', 'motorized', 'slow'], ['#1D2F6F', '#8390FA', '#6EAF46']))
color

In [ ]:
def plot_modal_split_comparison(name_df, modal_split_df):

    fig, ax = plt.subplots(figsize=(12, 8))
    modal_split_df.plot.bar(stacked=True, color=color, ax=ax)
    fs = 16
    plt.legend(loc="upper right", framealpha=1, fontsize=fs)
    plt.xlabel("Dataset", fontsize=fs)
    plt.ylabel("Modal split", fontsize=fs)
    plt.xticks(fontsize=fs)
    plt.tight_layout()
    
    plt.tight_layout()
    plt.savefig(os.path.join(out_path, f"modal_split_all_{name_df}.pdf"), bbox_inches='tight')
    plt.show()
    

In [ ]:
# plotting loop
for name_df, modal_split_df in zip(["count", "duration", "distance"],[modal_split_count, modal_split_duration, modal_split_distance]):
    plot_modal_split_comparison(name_df, modal_split_df)

## 7) Home / Work labeling

#### Collect work and home staypoints for GC1 and GC2

#### Compute staypoint purpose for Geolife and Yumuv

In [ ]:
yumuv_geolife_flag = staypoints['study'].isin(['Geolife', 'Yumuv'])
staypoints_wo_purpose = staypoints[yumuv_geolife_flag].copy()

pre_filter_kwargs = {'agg_level': 'dataset',
    'thresh_sp' : 0,
'thresh_loc' : 0,
'thresh_sp_at_loc' : 0,
'thresh_loc_time' : "0h",
'thresh_loc_period' : "0h"}

In [ ]:
staypoints_wo_purpose = ti.analysis.location_identifier(staypoints_wo_purpose, method="FREQ", **pre_filter_kwargs)
staypoints.activity_label.unique()


In [ ]:
sp_home = staypoints[staypoints['activity_label'] == 'home']
sp_work = staypoints[staypoints['activity_label'] == 'work']

#### Plotting

In [ ]:
#for sp_purpose, name in zip([sp_home, sp_work], ["home", "work"]):
sp_study = sp_home.copy()
name = 'home'

home_slots = []


# init result array
minute_slots = np.zeros(24 * 60)

# convert timestamp columns
sp_study["started_at"] = pd.to_datetime(sp_study["started_at"], utc=True)
sp_study["finished_at"] = pd.to_datetime(sp_study["finished_at"], utc=True)
# geolife flag
geolife_flag = sp_study['study'] == 'Geolife'

#sp_study.loc[geolife_flag, "started_at"]  = sp_study.loc[geolife_flag, "started_at"].dt.tz_convert("Europe/Paris")
#sp_study.loc[geolife_flag, "finished_at"]  = sp_study.loc[geolife_flag, "finished_at"].dt.tz_convert("Europe/Paris")

sp_study.loc[:, "started_at"]  = sp_study.loc[:, "started_at"].dt.tz_convert("Europe/Paris")
sp_study.loc[:, "finished_at"]  = sp_study.loc[:, "finished_at"].dt.tz_convert("Europe/Paris")




In [ ]:
#pd.set_option('display.max_colwidth', None)
#sp_study_.sort_index()['date']

In [ ]:
sp_study.finished_at.max()
t_start = sp_study.started_at
t_end = sp_study.finished_at
sp_study_ = sp_study[['started_at', 'finished_at', 'study']].copy()
#sp_study['date_range'] = df.apply(lambda x: pd.date_range(x["start_date"], x["end_date"]), axis=1)

sp_study_["date"] = sp_study_.apply(lambda x: pd.date_range(x["started_at"], x["finished_at"],  freq='min'), axis=1)
sp_study_expl = sp_study_.explode("date", ignore_index=True).drop(columns=["started_at", "finished_at"])


In [ ]:
count_by_minute = pd.DataFrame(sp_study_expl.groupby(['study', sp_study_expl.date.dt.hour, sp_study_expl.date.dt.minute]).size())
count_by_minute.index.rename(['study', 'hour', 'minute'], inplace=True)
count_by_minute.reset_index(inplace=True)
count_by_minute['minute_of_day'] = count_by_minute['hour'] * 60 + count_by_minute['minute']

count_by_minute_pivot = count_by_minute.pivot(columns='study', index='minute_of_day', values=0)
count_by_minute_pivot.plot()
plt.xticks(np.arange(0, len(minute_slots), 120), np.arange(0,24,2))
plt.legend(loc="lower left")
plt.ylabel=(f"Fraction of {name} staypoints covering time point")
plt.xlabel("Hour of the day")
# plt.savefig(os.path.join(out_path, f"plot_activity_distribution_{name}.pdf"))
plt.show()


In [ ]:
fig, ax = plt.subplots(figsize=(12,4))
home_slots = count_by_minute_pivot.values.transpose()
plt.imshow(home_slots, aspect=100)
for i in range(4):
    plt.plot([0, home_slots.shape[1]], [i+.5, i+.5], c="black", lw=1)
plt.xlim(0, 24)
plt.yticks(np.arange(4), map(lambda x: study_mapping[x], studies))
plt.xlabel("Hour of the day")
ax.xaxis.tick_top()
ax.xaxis.tickpad = -2
ax.xaxis.set_label_position("top")
plt.xticks(np.arange(0, len(minute_slots), 120), np.arange(0,24, 2))
# ax.set_xticklabels(np.arange(0, len(minute_slots), 120), np.arange(0,24, 2))
cbar = plt.colorbar(orientation="horizontal", ax=ax, aspect=16) #anchor=(-1, -1))
cbar.set_label(f"Fraction of {name} staypoints at time t")
plt.tight_layout()
# plt.savefig(os.path.join(out_path, f"imshow_act_distribution_{name}.pdf"), bbox_inches='tight')
plt.show()

## 8) Single user visualization

In [ ]:
from shapely.geometry import Point, Polygon

In [ ]:
x_min = 116.2
x_max = 116.5
y_min = 39.8
y_max = 40.1
sw = Point(x_min, y_min)
se = Point(x_max, y_min)
ne = Point(x_max, y_max)
nw = Point(x_min, y_max)

study_area = gpd.GeoDataFrame(columns=['geometry'], data=[Polygon([sw, se, ne, nw])], geometry='geometry', crs="EPSG:4326")



In [ ]:
# function to limit the spatial extend of the shown region
def get_in_range(linestring):
    x, y = linestring.xy
    x = np.array(x)
    y = np.array(y)
    if all(y>39.8) and all(y< 41): # 9.8, 9.3, 4.6, # all(x < 9.8) and all(x>9.3)
        return True
    return False

#trackintel.preprocessing.filter.spatial_filter(source, areas, method='within', re_project=False)

In [ ]:
# Load Geolife data of one user
tpls_vis = triplegs[triplegs['user_id']==51].copy()
tpls_vis = ti.preprocessing.filter.spatial_filter(triplegs, study_area, method='within')

In [ ]:
# Get staypoints as well
sp_vis = staypoints[staypoints['user_id']==51].copy()

In [ ]:
sp_vis = ti.preprocessing.filter.spatial_filter(sp_vis, study_area, method='within')

In [ ]:
sql = f"SELECT * FROM geolife.positionfixes WHERE user_id=51"
pfs_vis = ti.io.read_positionfixes_postgis(sql, con, index_col="id")
pfs_vis = ti.preprocessing.filter.spatial_filter(pfs_vis, study_area, method='within')

In [ ]:
ti.visualization.plot_triplegs(tpls_vis, 
                              staypoints= sp_vis,
                               plot_osm=True,
                                positionfixes=pfs_vis,
                              staypoints_radius=150,
                              out_filename=os.path.join(out_path, "geolife_tpls_sp_pfs"))

#### Plot only positionfixes

In [ ]:
ti.visualization.plot_positionfixes(pfs_vis,
                                   plot_osm=True,
                                   out_filename=os.path.join(out_path, "geolife_pfs")
                                   )

#### Plot staypoints with positionfixes

In [ ]:
ti.visualization.plot_staypoints(sp_vis,
    positionfixes=pfs_vis,
                                plot_osm=True,
                                 radius=150,
                                out_filename=os.path.join(out_path, "geolife_sp_pfs"))

### 8.2) Modal split visualization

In [ ]:
# Use GC2 user

tpls_vis = triplegs[triplegs['user_id']=='c9aa08e2-1a5d-4d41-ae62-6110a9072b23']

In [ ]:
# rename to mode
tpls_vis["mode"] = tpls_vis["mode_detected"]

In [ ]:
# compute modal split
modal_split = calculate_modal_split(tpls_vis, freq="M", metric='count', per_user=False, norm=True)

In [ ]:
# order the transport modes for colour choices
column_order = ["Mode::Car", "Mode::Bicycle", "Mode::Walk", "Mode::Bus", "Mode::Train", "Mode::Ecar", "Mode::Ebicycle", "Mode::Airplane"]
modal_split = modal_split[column_order]

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
ax = ti.visualization.modal_split.plot_modal_split(modal_split, 
                                              date_fmt_x_axis='%b',
                                              y_label='Percentage of daily count',
                                              skip_xticks=0,
                                              n_col_legend=4,
                                              fs=20,
                                              axis=ax,
                                             borderaxespad=2
                                             )
fig.autofmt_xdate()
plt.savefig(os.path.join(out_path, "modal_split_count"))

#### Same for distance

In [ ]:
modal_split = calculate_modal_split(tpls_vis, freq="M", metric='distance', per_user=False, norm=True)
modal_split = modal_split[column_order]

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
ax = ti.visualization.modal_split.plot_modal_split(modal_split, 
                                              date_fmt_x_axis='%b',
                                              y_label='Percentage of daily distance',
                                              skip_xticks=0,
                                              n_col_legend=4,
                                              fs=20,
                                              axis=ax,
                                             borderaxespad=2
                                             )
fig.autofmt_xdate()
plt.savefig(os.path.join(out_path, "modal_split_distance"))